# 8차시 실습: 가드레일 기초 - OpenAI Moderation API 활용

이번 시간에는 LLM을 안전하게 보호하는 첫 번째 관문인 **가드레일(Guardrails)**을 실습합니다.
OpenAI에서 무료로 제공하는 `Moderation API`를 사용하여 사용자 입력의 유해성을 판별하고, 점수(Score) 기반으로 제어하는 방법을 배웁니다.

### 학습 목표
1. Moderation API 호출 및 JSON 응답 구조 분석
2. `Flagged`와 `Score`의 차이점 이해 및 임계값(Threshold) 튜닝
3. 단순 가드레일의 한계점(False Positive) 체험

In [ ]:
# 필수 라이브러리 설치
!pip install openai pandas matplotlib

In [ ]:
import os
from openai import OpenAI
import pandas as pd
import matplotlib.pyplot as plt

# OpenAI API 키 설정 (Moderation API는 OpenAI 계정이 필요합니다)

os.environ["OPENAI_API_KEY"] = "API Key를 넣으세요!"

client = OpenAI()

def check_moderation(text):
    """
    OpenAI Moderation API를 호출하여 유해성 결과를 반환합니다.
    """
    response = client.moderations.create(input=text)
    return response.results[0]

## 1. 기본 검사: 안전한 문장 vs 위험한 문장
API가 문장을 어떻게 분류하는지 확인해 봅시다.

In [2]:
# 테스트 케이스
safe_text = "안녕하세요! 오늘 날씨가 참 좋네요. 점심 메뉴 추천해 주실 수 있나요?"
unsafe_text = "저 녀석을 때려주고 싶어. 아주 잔인하게 복수할 거야." # (교육용 예시)

# 1. 안전한 문장 검사
res_safe = check_moderation(safe_text)
print(f"[Safe Text] Flagged: {res_safe.flagged}")

# 2. 위험한 문장 검사
res_unsafe = check_moderation(unsafe_text)
print(f"[Unsafe Text] Flagged: {res_unsafe.flagged}")

# 상세 점수 출력
print("\n--- 위험 문장 상세 점수 ---")
for category, score in res_unsafe.category_scores.to_dict().items():
    if score > 0.01: # 0.01점 이상만 출력
        print(f"{category}: {score:.4f}")

[Safe Text] Flagged: False
[Unsafe Text] Flagged: True

--- 위험 문장 상세 점수 ---
harassment: 0.3417
harassment/threatening: 0.3228
illicit: 0.0947
illicit/violent: 0.0578
violence: 0.9430


## 2. 임계값(Threshold) 튜닝 실험
API가 기본적으로 제공하는 `flagged` 값은 보수적일 수 있습니다.
서비스 성격에 따라 우리가 직접 점수(Score)를 기준으로 차단 여부를 결정해 봅시다.

In [3]:
def custom_guardrail(text, threshold=0.5):
    """
    사용자 정의 임계값을 적용한 가드레일 함수
    """
    result = check_moderation(text)
    scores = result.category_scores.to_dict()
    
    # 가장 높은 점수를 가진 카테고리 찾기
    max_category = max(scores, key=scores.get)
    max_score = scores[max_category]
    
    print(f"입력: '{text}'")
    print(f" -> 최대 위험 항목: {max_category} ({max_score:.4f})")
    
    if max_score > threshold:
        print(f"🚫 [차단] 기준치({threshold}) 초과!\n")
        return False
    else:
        print(f"✅ [통과] 기준치({threshold}) 미만.\n")
        return True

# 애매한 문장 테스트 (폭력성이 있지만 농담일 수 있는 경우)
ambiguous_text = "이 영화 완전 죽여주네! 진짜 미친 액션이야!"

# 1. 엄격한 기준 (Strict: 0.1)
custom_guardrail(ambiguous_text, threshold=0.1)

# 2. 느슨한 기준 (Loose: 0.8)
custom_guardrail(ambiguous_text, threshold=0.8)

입력: '이 영화 완전 죽여주네! 진짜 미친 액션이야!'
 -> 최대 위험 항목: violence (0.2037)
🚫 [차단] 기준치(0.1) 초과!

입력: '이 영화 완전 죽여주네! 진짜 미친 액션이야!'
 -> 최대 위험 항목: violence (0.2037)
✅ [통과] 기준치(0.8) 미만.



True

## 3. Moderation API의 한계 (False Positive)
단순 텍스트 분류 모델은 **문맥(Context)**을 완벽히 이해하지 못할 수 있습니다.
특히 IT 기술 용어나 소설 창작 상황에서 발생하는 **오탐(False Positive)** 사례를 확인해 봅니다.

In [6]:
# IT 기술 용어 (프로세스 종료)
tech_context = "리눅스 서버에서 좀비 프로세스를 죽이는(kill) 명령어를 알려줘."

# 소설 창작 상황
novel_context = "소설 속 악당이 주인공을 위협하는 대사를 써줘: '너를 가만두지 않겠다.'"

print("--- 문맥 오해 테스트 ---")
custom_guardrail(tech_context, threshold=0.3) 
custom_guardrail(novel_context, threshold=0.3)

--- 문맥 오해 테스트 ---
입력: '리눅스 서버에서 좀비 프로세스를 죽이는(kill) 명령어를 알려줘.'
 -> 최대 위험 항목: violence (0.4269)
🚫 [차단] 기준치(0.3) 초과!

입력: '소설 속 악당이 주인공을 위협하는 대사를 써줘: '너를 가만두지 않겠다.''
 -> 최대 위험 항목: violence (0.5250)
🚫 [차단] 기준치(0.3) 초과!



False

In [7]:
import re

class FinancialSafetyGateway:
    def __init__(self):
        # 금융사 금지 단어/패턴 (정규식)
        self.blocked_patterns = [
            r"무조건 수익", r"원금 보장", r"100% 상승",  # 불법 투자 권유
            r"내부등급", r"특수코드",                   # 내부 기밀
        ]
        # PII 패턴 (주민번호 예시)
        self.pii_pattern = r"\d{6}-[1-4]\d{6}"

    def input_filter(self, user_input: str) -> str:
        """입력 단계: PII 마스킹"""
        if re.search(self.pii_pattern, user_input):
            print("🛡️ [Gateway] 입력에서 PII 감지! 마스킹 처리합니다.")
            return re.sub(self.pii_pattern, "******-*******", user_input)
        return user_input

    def output_filter(self, llm_response: str) -> str:
        """출력 단계: 금융 컴플라이언스 위반 검사"""
        for pattern in self.blocked_patterns:
            if re.search(pattern, llm_response):
                print(f"🚫 [Gateway] 정책 위반 표현 감지: '{pattern}'")
                return "⚠️ [차단됨] 금융소비자보호법 및 내부 정책에 따라 제공할 수 없는 답변입니다."
        return llm_response

# --- 테스트 ---
gateway = FinancialSafetyGateway()

# 1. 입력 필터 테스트 (PII)
user_query = "제 주민번호 900101-1234567로 조회해주세요."
safe_query = gateway.input_filter(user_query)
print(f"LLM 입력값: {safe_query}")

# 2. 출력 필터 테스트 (불법 권유)
unsafe_response = "이 상품은 무조건 수익이 나는 확실한 종목입니다."
final_output = gateway.output_filter(unsafe_response)
print(f"사용자 응답: {final_output}")

🛡️ [Gateway] 입력에서 PII 감지! 마스킹 처리합니다.
LLM 입력값: 제 주민번호 ******-*******로 조회해주세요.
🚫 [Gateway] 정책 위반 표현 감지: '무조건 수익'
사용자 응답: ⚠️ [차단됨] 금융소비자보호법 및 내부 정책에 따라 제공할 수 없는 답변입니다.


In [11]:
import time
import json

class HybridSafetyGateway(FinancialSafetyGateway):
    """패턴 기반 필터 + Moderation API를 결합한 하이브리드 가드레일"""
    
    def __init__(self):
        super().__init__()
        self.client = client
        
    def check_moderation_api(self, text: str) -> dict:
        """Moderation API 호출"""
        try:
            response = self.client.moderations.create(input=text)
            return {
                "flagged": response.results[0].flagged,
                "scores": response.results[0].category_scores.to_dict()
            }
        except Exception as e:
            return {"error": str(e)}
    
    def call_llm(self, text: str) -> str:
        """LLM API 호출 (간단한 응답 생성)"""
        try:
            response = self.client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": text}],
                max_tokens=50,
                temperature=0.7
            )
            return response.choices[0].message.content
        except Exception as e:
            return f"Error: {str(e)}"

    # --- 1. 패턴 필터 + LLM API ---
    def pattern_then_llm(self, user_input: str) -> tuple[bool, str, dict]:
        """패턴 필터 적용 후 LLM 호출"""
        timing = {}
        
        # 입력 패턴 필터
        start = time.perf_counter()
        processed = self.input_filter(user_input)
        timing["input_pattern_filter_ms"] = (time.perf_counter() - start) * 1000
        
        # LLM 호출
        start = time.perf_counter()
        llm_response = self.call_llm(processed)
        timing["llm_ms"] = (time.perf_counter() - start) * 1000
        
        # 출력 패턴 필터
        start = time.perf_counter()
        final_output = self.output_filter(llm_response)
        timing["output_pattern_filter_ms"] = (time.perf_counter() - start) * 1000
        
        timing["total_ms"] = sum([v for k, v in timing.items() if "ms" in k])
        return True, final_output, timing

    # --- 2. LLM API만 사용 (가드레일 없음) ---
    def llm_only(self, user_input: str) -> tuple[str, dict]:
        """LLM만 호출 (가드레일 없음)"""
        timing = {}
        
        start = time.perf_counter()
        llm_response = self.call_llm(user_input)
        timing["llm_ms"] = (time.perf_counter() - start) * 1000
        timing["total_ms"] = timing["llm_ms"]
        
        return llm_response, timing

    # --- 3. 모든 가드레일 적용 (패턴 + Moderation + LLM) ---
    def full_pipeline(self, user_input: str) -> tuple[str, dict]:
        """모든 가드레일 적용: 패턴필터 → Moderation → LLM → 패턴필터 → Moderation"""
        timing = {}
        
        # 입력 패턴 필터
        start = time.perf_counter()
        processed = self.input_filter(user_input)
        timing["input_pattern_filter_ms"] = (time.perf_counter() - start) * 1000
        
        # 입력 Moderation API
        start = time.perf_counter()
        mod_result = self.check_moderation_api(processed)
        timing["input_moderation_ms"] = (time.perf_counter() - start) * 1000
        
        # LLM 호출
        start = time.perf_counter()
        llm_response = self.call_llm(processed)
        timing["llm_ms"] = (time.perf_counter() - start) * 1000
        
        # 출력 패턴 필터
        start = time.perf_counter()
        checked = self.output_filter(llm_response)
        timing["output_pattern_filter_ms"] = (time.perf_counter() - start) * 1000
        
        # 출력 Moderation API
        start = time.perf_counter()
        out_mod_result = self.check_moderation_api(checked)
        timing["output_moderation_ms"] = (time.perf_counter() - start) * 1000
        
        timing["total_ms"] = sum([v for k, v in timing.items() if "ms" in k])
        return checked, timing

# --- 벤치마크 ---
print("=" * 80)
print("LLM 가드레일 성능 평가 (3가지 방식 비교)")
print("=" * 80)

gateway_hybrid = HybridSafetyGateway()

# 테스트 케이스
test_cases = [
    ("정상 요청", "주식 투자 시 주의사항을 알려줘"),
    ("PII 포함", "내 주민번호 900101-1234567로 계좌를 조회해줘"),
    ("정책 위반", "이 상품은 무조건 수익이 나는 확실한 종목입니다."),
]

results_data = []

print("\n[방식 1] 패턴 필터 + LLM API")
print("-" * 80)
for category, text in test_cases:
    _, output, timing = gateway_hybrid.pattern_then_llm(text)
    print(f"{category:<15} → {timing['total_ms']:>8.2f}ms (패턴: {timing['input_pattern_filter_ms']:.2f}ms + LLM: {timing['llm_ms']:.2f}ms + 패턴: {timing['output_pattern_filter_ms']:.2f}ms)")
    results_data.append({
        "방식": "패턴 필터 + LLM",
        "카테고리": category,
        "총합(ms)": timing['total_ms']
    })

print("\n[방식 2] LLM API (가드레일 없음)")
print("-" * 80)
for category, text in test_cases:
    output, timing = gateway_hybrid.llm_only(text)
    print(f"{category:<15} → {timing['total_ms']:>8.2f}ms (LLM: {timing['llm_ms']:.2f}ms)")
    results_data.append({
        "방식": "LLM만 사용",
        "카테고리": category,
        "총합(ms)": timing['total_ms']
    })

print("\n[방식 3] 패턴 필터 + Moderation + LLM + 패턴 필터 + Moderation")
print("-" * 80)
for category, text in test_cases:
    output, timing = gateway_hybrid.full_pipeline(text)
    print(f"{category:<15} → {timing['total_ms']:>8.2f}ms (입력패턴: {timing['input_pattern_filter_ms']:.2f}ms + 입력API: {timing['input_moderation_ms']:.2f}ms + LLM: {timing['llm_ms']:.2f}ms + 출력패턴: {timing['output_pattern_filter_ms']:.2f}ms + 출력API: {timing['output_moderation_ms']:.2f}ms)")
    results_data.append({
        "방식": "전체 가드레일",
        "카테고리": category,
        "총합(ms)": timing['total_ms']
    })

# 결과 테이블
print("\n" + "=" * 80)
print("📊 성능 비교 요약")
print("=" * 80)

df_results = pd.DataFrame(results_data)

# 방식별로 피벗
pivot_table = df_results.pivot(index="카테고리", columns="방식", values="총합(ms)")
pivot_table = pivot_table[["패턴 필터 + LLM", "LLM만 사용", "전체 가드레일"]]
print("\n" + pivot_table.to_string())

print("\n" + "=" * 80)
print("평균값 비교")
print("=" * 80)
avg_summary = df_results.groupby("방식")["총합(ms)"].mean()
avg_summary = avg_summary[["패턴 필터 + LLM", "LLM만 사용", "전체 가드레일"]]
print("\n" + avg_summary.to_string())


LLM 가드레일 성능 평가 (3가지 방식 비교)

[방식 1] 패턴 필터 + LLM API
--------------------------------------------------------------------------------
정상 요청           →  1918.58ms (패턴: 0.02ms + LLM: 1918.53ms + 패턴: 0.03ms)
🛡️ [Gateway] 입력에서 PII 감지! 마스킹 처리합니다.
PII 포함          →  1548.26ms (패턴: 0.05ms + LLM: 1548.19ms + 패턴: 0.02ms)
정책 위반           →  1184.91ms (패턴: 0.01ms + LLM: 1184.88ms + 패턴: 0.02ms)

[방식 2] LLM API (가드레일 없음)
--------------------------------------------------------------------------------
정상 요청           →  1635.16ms (LLM: 1635.16ms)
PII 포함          →  1026.23ms (LLM: 1026.23ms)
정책 위반           →   876.29ms (LLM: 876.29ms)

[방식 3] 패턴 필터 + Moderation + LLM + 패턴 필터 + Moderation
--------------------------------------------------------------------------------
정상 요청           →  2342.04ms (입력패턴: 0.02ms + 입력API: 259.99ms + LLM: 1673.84ms + 출력패턴: 0.02ms + 출력API: 408.18ms)
🛡️ [Gateway] 입력에서 PII 감지! 마스킹 처리합니다.
PII 포함          →  2473.87ms (입력패턴: 0.04ms + 입력API: 309.06ms + LLM: 817.58ms + 출력패턴: 0.